In [4]:
import csv
import pandas as pd
import emoji
import math
import advertools as adv
import numpy as np
from numpy import array
from numpy.linalg import norm
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
countries=['France_tweets_of_interest.csv','Germany_tweets_of_interest.csv','Iceland_tweets_of_interest.csv','Iran_tweets_of_interest.csv',
          'Japan_tweets_of_interest.csv','Mexico_tweets_of_interest.csv','Morocco_tweets_of_interest.csv','Nigeria_tweets_of_interest.csv'
          ,'Panama_tweets_of_interest.csv','Poland_tweets_of_interest.csv','Peru_tweets_of_interest.csv','Portugal_tweets_of_interest.csv',
          'Russia_tweets_of_interest.csv','Serbia_tweets_of_interest.csv','Senegal_tweets_of_interest.csv','South_korea_tweets_of_interest.csv',
          'Spain_tweets_of_interest.csv','Sweden_tweets_of_interest.csv','Switzerland_tweets_of_interest.csv','Tunisia_tweets_of_interest.csv',
          'Uruguay_tweets_of_interest.csv','Nigeria_tweets_of_interest.csv']

In [ ]:
rem_22=['France','Germany','Iceland','Iran','Japan','Mexico','Morocco','Nigeria','Panama','Poland','Peru','Portugal','Russia','Serbia',
       'Senegal','South_korea','Spain','Sweden','Switzerland','Tunisia','Uruguay','Nigeria']

In [3]:
def fun(dataset):
    country=[]
    with open(dataset,encoding="utf-8") as file:
        for line in file:
            country.append(line.split('|@|||$|')[2])
            
    # getting the tweets with emojis
    country_emoji=[]
    emoji_count=[]
    for i in country:
        if len(emoji.emoji_list(i))!=0:
            country_emoji.append(i)
            emoji_count.append((emoji.emoji_list(i)[0]['emoji']))
            
    
    tweets_length=len(country_emoji)
    country_emoji_summary=adv.extract_emoji(country_emoji)
    
    #creating frequency matrix for face emojis
    emoji_index=pd.Series.to_dict(adv.emoji_df[:120]['emoji'])
    emoji_index = dict([(value, key) for key, value in emoji_index.items()])
    
    
    p=[]
    for i in country_emoji_summary['emoji']:
        l=[0]*120
        for j in i:
            if j in emoji_index.keys():
                l[emoji_index[j]-1]=1
        p.append(l)
    # creating a dataframe
    emoji_matrix=pd.DataFrame(p,columns=emoji_index.keys())
    
    #term frequency tf
    tf_series=emoji_matrix.sum(axis='rows')
    
    # iverse term frequency
    idf_series=[]
    for i in tf_series:
        if i!=0:
            idf_series.append((math.log(tweets_length/i))+1)
        else:
            idf_series.append(0)
    idf_series
    
    #tf*idf 
    tf_idf_series=tf_series*idf_series
    
    # euclidiean form of vector
    emoji_norm=norm(tf_idf_series)
    nomralized_frequency=[]
    for i in tf_idf_series:
        nomralized_frequency.append(i/emoji_norm)
        
        
    
    return nomralized_frequency
    
    
    
    
    

In [43]:
df10=pd.read_csv('top_10_countries.csv')

In [61]:
df10.rename(columns={df10.columns[0]: 'Country'},inplace=True)

In [7]:
remaining_countries=[]

In [40]:
for i in range(len(countries)):
    remaining_countries.append(fun(countries[i]))

In [114]:
# rem22_dic['Nigeria']=remaining_countries[21]

In [108]:
# adding the vectors generated for each country to the dictonary
rem22_dic={}
c=0
for i in remaining_countries:
    #print(rem_22[c])
    rem22_dic[rem_22[c]]=i
    
    c+=1
    

France
Germany
Iceland
Iran
Japan
Mexico
Morocco
Nigeria
Panama
Poland
Peru
Portugal
Russia
Serbia
Senegal
South_korea
Spain
Sweden
Switzerland
Tunisia
Uruguay
Nigeria


In [110]:
# Joining 10 and 22 countries
for i,idx in df10.iterrows():
    #print(idx['Country'],idx[1:120])
    rem22_dic[idx['Country']]=list(idx[1:121].values)
    

In [117]:
top31=pd.DataFrame.from_dict(rem22_dic,orient='index')

In [118]:
top31

,0,1,2,3,4,5,6,7,8,9,...,110,111,112,113,114,115,116,117,118,119
France,0.065608,0.048864,0.111636,0.030531,0.097068,0.148114,0.570595,0.059741,0.047456,0.264379,...,0.002568,0.004011,0.012367,0.025772,0.002881,0.005669,0.003582,0.004307,0.001004,0.070265
Germany,0.063282,0.072312,0.171659,0.060696,0.118366,0.183265,0.572640,0.062853,0.054200,0.280114,...,0.002043,0.002496,0.006643,0.017168,0.001256,0.003085,0.004499,0.003942,0.000402,0.170945
Iceland,0.052212,0.074689,0.111235,0.072275,0.162819,0.126243,0.623191,0.067389,0.104641,0.220732,...,0.008143,0.011646,0.004396,0.036022,0.008143,0.004396,0.008143,0.000000,0.000000,0.126243
Iran,0.041811,0.108628,0.246263,0.046111,0.158072,0.091080,0.690462,0.046111,0.028261,0.115419,...,0.001375,0.004763,0.007813,0.024274,0.001982,0.004763,0.004230,0.001375,0.000000,0.083189
Japan,0.254503,0.139997,0.190330,0.300748,0.285680,0.170337,0.262163,0.052257,0.026516,0.111742,...,0.026296,0.024161,0.013552,0.025535,0.005023,0.009286,0.010784,0.011012,0.001809,0.067854
Mexico,0.073810,0.044908,0.116481,0.045638,0.122208,0.232333,0.516049,0.045609,0.061326,0.156079,...,0.003327,0.003907,0.012114,0.018058,0.003507,0.005077,0.008471,0.004777,0.002027,0.061545
Morocco,0.031267,0.031267,0.101592,0.049306,0.089351,0.168435,0.761233,0.028193,0.032781,0.149492,...,0.000000,0.001365,0.021829,0.019353,0.000000,0.000000,0.002547,0.006765,0.000000,0.047229
Nigeria,0.042858,0.046560,0.171904,0.045396,0.065081,0.238678,0.803873,0.035226,0.019342,0.058217,...,0.000708,0.001327,0.030705,0.014139,0.001655,0.001057,0.002038,0.002780,0.000708,0.079482
Panama,0.044809,0.043969,0.107597,0.044529,0.098822,0.319752,0.626424,0.074323,0.046894,0.114963,...,0.000843,0.002045,0.010434,0.022829,0.001814,0.001094,0.003593,0.002723,0.000843,0.035803
Poland,0.072541,0.073955,0.180944,0.052275,0.073602,0.281172,0.580518,0.118044,0.055329,0.376171,...,0.001758,0.003479,0.007741,0.010082,0.004693,0.003685,0.006624,0.003685,0.000817,0.148346


In [120]:
top31.to_csv('top31.csv')